In [ ]:
### Data
import xarray as xr
from netCDF4 import Dataset
import numpy as np
from wrf import getvar, interplevel, smooth2d

### Plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.ticker as mticker
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import matplotlib.transforms as mtransforms

### Warnings
import warnings
warnings.filterwarnings('ignore')

CP = 1005.7
RD = 287.04
P0 = 1000.
TR = 300.
LV = 2.501e6
EPS = 1.


In [ ]:
### Parameters
nx = 400
ny = 250
nz = 44
ti = 0
lev = 600

### Read in the data
ctl = Dataset('/p/work1/lloveras/nov2018/30km_files/ctl/wrfout_d01_2018-11-13_12_00_00')
full = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_full/wrfout_d01_2018-11-13_12_00_00')
large = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_large/wrfout_d01_2018-11-13_12_00_00')
small = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_small/wrfout_d01_2018-11-13_12_00_00')
box = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_box/wrfout_d01_2018-11-13_12_00_00')
hole = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_hole/wrfout_d01_2018-11-13_12_00_00')

### Latitude and longitude
lats = np.asarray(getvar(ctl,'lat',timeidx=ti))
lons = np.asarray(getvar(ctl,'lon',timeidx=ti))
lons[lons > 0] -= 360

### Pressure
p_ctl = np.asarray(getvar(ctl,'pressure',timeidx=ti))
p_full = np.asarray(getvar(full,'pressure',timeidx=ti))
p_large = np.asarray(getvar(large,'pressure',timeidx=ti))
p_small = np.asarray(getvar(small,'pressure',timeidx=ti))
p_box = np.asarray(getvar(box,'pressure',timeidx=ti))
p_hole = np.asarray(getvar(hole,'pressure',timeidx=ti))

### Height
z_ctl = np.asarray(getvar(ctl,'z',timeidx=ti))
zl_ctl = np.asarray(interplevel(z_ctl, p_ctl, lev))

### U
u_ctl = np.asarray(getvar(ctl,'ua'))
u_full = np.asarray(getvar(full,'ua'))
u_large = np.asarray(getvar(large,'ua'))
u_small = np.asarray(getvar(small,'ua'))
u_box = np.asarray(getvar(box,'ua'))
u_hole = np.asarray(getvar(hole,'ua'))

ul_ctl = interplevel(u_ctl, p_ctl, lev)
du_full = interplevel(u_full, p_full, lev) - ul_ctl
du_large = u_large[23,:,:] - u_ctl[23,:,:]
du_small = u_small[23,:,:] - u_ctl[23,:,:]
du_box = interplevel(u_box, p_box, lev) - ul_ctl
du_hole = interplevel(u_hole, p_hole, lev) - ul_ctl

### V
v_ctl = np.asarray(getvar(ctl,'va'))
v_full = np.asarray(getvar(full,'va'))
v_large = np.asarray(getvar(large,'va'))
v_small = np.asarray(getvar(small,'va'))
v_box = np.asarray(getvar(box,'va'))
v_hole = np.asarray(getvar(hole,'va'))

vl_ctl = interplevel(v_ctl, p_ctl, lev)
dv_full = interplevel(v_full, p_full, lev) - vl_ctl
dv_large = v_large[23,:,:] - v_ctl[23,:,:]
dv_small = v_small[23,:,:] - v_ctl[23,:,:]
dv_box = interplevel(v_box, p_box, lev) - vl_ctl
dv_hole = interplevel(v_hole, p_hole, lev) - vl_ctl

### T
t_ctl = np.asarray(getvar(ctl,'THM')) + TR
t_full = np.asarray(getvar(full,'THM')) + TR
t_large = np.asarray(getvar(large,'THM')) + TR
t_small = np.asarray(getvar(small,'THM')) + TR
t_box = np.asarray(getvar(box,'THM')) + TR
t_hole = np.asarray(getvar(hole,'THM')) + TR

tk_ctl = t_ctl/((P0/p_ctl)**(RD/CP))
tk_full = t_full/((P0/p_full)**(RD/CP))
tk_large = t_large/((P0/p_large)**(RD/CP))
tk_small = t_small/((P0/p_small)**(RD/CP))
tk_box = t_box/((P0/p_box)**(RD/CP))
tk_hole = t_hole/((P0/p_hole)**(RD/CP))

tl_ctl = interplevel(tk_ctl, p_ctl, lev)
dt_full = interplevel(tk_full, p_full, lev) - tl_ctl
dt_large = (tk_large[24,:,:] + tk_large[23,:,:])/2 - (tk_ctl[24,:,:] + tk_ctl[23,:,:])/2
dt_small = (tk_small[24,:,:] + tk_small[23,:,:])/2 - (tk_ctl[24,:,:] + tk_ctl[23,:,:])/2
dt_box = interplevel(tk_box, p_box, lev) - tl_ctl
dt_hole = interplevel(tk_hole, p_hole, lev) - tl_ctl

### DTE
dte_full = np.sqrt(0.5*(du_full**2 + dv_full**2 + (CP/TR)*dt_full**2))
dte_large = np.sqrt(0.5*(du_large**2 + dv_large**2 + (CP/TR)*dt_large**2))
dte_small = np.sqrt(0.5*(du_small**2 + dv_small**2 + (CP/TR)*dt_small**2))
dte_box = np.sqrt(0.5*(du_box**2 + dv_box**2 + (CP/TR)*dt_box**2))
dte_hole = np.sqrt(0.5*(du_hole**2 + dv_hole**2 + (CP/TR)*dt_hole**2))

In [ ]:
### Contour interval
zlvls = np.arange(0,3.25,0.25)

cmaplist = plt.cm.turbo(np.linspace(0.2, 1, len(zlvls) + 1))
cmaplist[0] = (1.0,1.0,1.0,1.0)
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('mcm',cmaplist, len(zlvls)+1)
cmap.set_over('k')

### Plot subdomain
min_lat = 15
max_lat = 60
min_lon = -125
max_lon = -75

buf = 25
zl_ctl[(lons < min_lon-buf) | (lons > max_lon+buf)] = np.NaN

### Cartopy projection
proj = crs.LambertConformal(central_longitude=-100,standard_parallels=(20,70))

fig, axd = plt.subplot_mosaic([['topleft','topmiddle','topright'],
                               ['bottomleft','bottommiddle','bottomright'],
                               ['cbar','cbar','cbar']],
                              constrained_layout=True, figsize=(6.8,4.6), dpi=200, 
                              gridspec_kw={'width_ratios':[1,1,1],'height_ratios':[1,1,0.05]},
                              per_subplot_kw={'topleft':{'projection':proj},
                                              'topmiddle':{'projection':proj},
                                              'topright':{'projection':proj},
                                              'bottomleft':{'projection':proj},
                                              'bottommiddle':{'projection':proj},
                                              'bottomright':{'projection':proj}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### TOP LEFT
#############

# Add the gridlines
gls = axd['topleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=False
gls.rotate_labels=False

cs1 = axd['topleft'].contour(lons, lats, smooth2d(zl_ctl,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['topleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topleft'].add_feature(states, linewidth=0.5, edgecolor='k',alpha=0.5)
axd['topleft'].coastlines('50m', linewidth=0.5,color='k',alpha=0.5)

# Title
axd['topleft'].set_title('Control')
axd['topleft'].text(0.0, 1.0, '(a)',transform=axd['topleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)
axd['topleft'].text(0.17, 0.55, 'No Perturbations',transform=axd['topleft'].transAxes + trans, fontsize=12,verticalalignment='top', bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP MIDDLE
##############

# Add the gridlines
gls = axd['topmiddle'].gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topmiddle'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])

cs1 = axd['topmiddle'].contour(lons, lats, smooth2d(zl_ctl,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['topmiddle'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im1 = axd['topmiddle'].contourf(lons, lats, dte_box, levels=zlvls,
            cmap = cmap, extend='max',
            transform=crs.PlateCarree())

min_lat1 = 25
max_lat1 = 40
min_lon1 = -107
max_lon1 = -92

axd['topmiddle'].plot([min_lon1, min_lon1, max_lon1, max_lon1, min_lon1], 
                    [min_lat1, max_lat1, max_lat1, min_lat1, min_lat1], 
         color='magenta', linewidth=1.5, marker='None',
         transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topmiddle'].add_feature(states, linewidth=0.5, edgecolor='k',alpha=0.5)
axd['topmiddle'].coastlines('50m', linewidth=0.5,color='k',alpha=0.5)

# Title
axd['topmiddle'].set_title('Target')
axd['topmiddle'].text(0.0, 1.0, '(b)',transform=axd['topmiddle'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP RIGHT
##############

# Add the gridlines
gls = axd['topright'].gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])

cs1 = axd['topright'].contour(lons, lats, smooth2d(zl_ctl,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['topright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im1 = axd['topright'].contourf(lons, lats, dte_small, levels=zlvls,
            cmap = cmap, extend='max',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topright'].add_feature(states, linewidth=0.5, edgecolor='k',alpha=0.5)
axd['topright'].coastlines('50m', linewidth=0.5,color='k',alpha=0.5)

# Title
axd['topright'].set_title('Short $\lambda$')
axd['topright'].text(0.0, 1.0, '(c)',transform=axd['topright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM LEFT
##############

# Add the gridlines
gls = axd['bottomleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.rotate_labels=False

cs1 = axd['bottomleft'].contour(lons, lats, smooth2d(zl_ctl,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['bottomleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im1 = axd['bottomleft'].contourf(lons, lats, dte_full, levels=zlvls,
            cmap = cmap, extend='max',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomleft'].add_feature(states, linewidth=0.5, edgecolor='k',alpha=0.5)
axd['bottomleft'].coastlines('50m', linewidth=0.5,color='k',alpha=0.5)

# Title
axd['bottomleft'].set_title('Full')
axd['bottomleft'].text(0.0, 1.0, '(d)',transform=axd['bottomleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM MIDDLE
##############

# Add the gridlines
gls = axd['bottommiddle'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottommiddle'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.rotate_labels=False
gls.left_labels=False


cs1 = axd['bottommiddle'].contour(lons, lats, smooth2d(zl_ctl,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['bottommiddle'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im1 = axd['bottommiddle'].contourf(lons, lats, dte_hole, levels=zlvls,
            cmap = cmap, extend='max',
            transform=crs.PlateCarree())

axd['bottommiddle'].plot([min_lon1, min_lon1, max_lon1, max_lon1, min_lon1], 
                    [min_lat1, max_lat1, max_lat1, min_lat1, min_lat1], 
         color='magenta', linewidth=1.5, marker='None',
         transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottommiddle'].add_feature(states, linewidth=0.5, edgecolor='k',alpha=0.5)
axd['bottommiddle'].coastlines('50m', linewidth=0.5,color='k',alpha=0.5)

# Title
axd['bottommiddle'].set_title('Target$^\complement$')
axd['bottommiddle'].text(0.0, 1.0, '(e)',transform=axd['bottommiddle'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM RIGHT
##############

# Add the gridlines
gls = axd['bottomright'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=False
gls.rotate_labels=False

cs1 = axd['bottomright'].contour(lons, lats, smooth2d(zl_ctl,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['bottomright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im1 = axd['bottomright'].contourf(lons, lats, dte_large, levels=zlvls,
            cmap = cmap, extend='max',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomright'].add_feature(states, linewidth=0.5, edgecolor='k',alpha=0.5)
axd['bottomright'].coastlines('50m', linewidth=0.5,color='k',alpha=0.5)

# Title
axd['bottomright'].set_title('Long $\lambda$')
axd['bottomright'].text(0.0, 1.0, '(f)',transform=axd['bottomright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### SET THE COLORBAR AND SHOW
cbar = fig.colorbar(im1, orientation='horizontal', cax=axd['cbar'])
plt.savefig('/p/work1/lloveras/nov2018/figs_pdf/fig07_wrf_setup.pdf',bbox_inches='tight')
plt.show()
